<a href="https://colab.research.google.com/github/loversky01/shopify/blob/master/Ph%C3%A2n%20t%C3%ADch%20c%E1%BA%A3m%20x%C3%BAc%20tr%C3%AAn%20Shopify%20%C4%90%C3%A1nh%20gi%C3%A1%20b%E1%BA%B1ng%20c%C3%A1ch%20s%E1%BB%AD%20d%E1%BB%A5ng%20Ph%C6%B0%C6%A1ng%20ph%C3%A1p%20ti%E1%BA%BFp%20c%E1%BA%ADn%20TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Lover Sky - Minh Vương 

In [3]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
#Đọc tập dữ liệu bằng cách sử dụng pandas

In [5]:
df=pd.read_csv('reviews.csv')

In [ ]:
#Xem trước tập dữ liệu

In [6]:
df.head(3)

,app_id,author,rating,posted_at,Sentence,helpful_count,developer_reply,developer_reply_posted_at
0,b1da53a4-0474-4700-9620-bf386bc033fb,Consuela,5,6-Aug-20,Great and super fast customer service! Highly ...,0,NaN,NaN
1,b1da53a4-0474-4700-9620-bf386bc033fb,L'Atelier Global,5,4-Aug-20,"Still setting up my store, and after initially...",0,NaN,NaN
2,b1da53a4-0474-4700-9620-bf386bc033fb,city'super E-Shop,5,4-Aug-20,"This is an excellent search app, which they ha...",0,NaN,NaN


In [ ]:
#Chúng ta chỉ yêu cầu các cột review_body, star_rating mô tả các bài đánh giá,
#xếp hạng theo sao của từng bài đánh giá tương ứng.

In [8]:
df=df[['Sentence','rating']]

In [ ]:
#Loại bỏ các giá trị Null, bị thiếu và đặt lại chỉ mục

In [9]:
df=df.dropna()
df = df.reset_index(drop=True)
df

,Sentence,rating
0,Great and super fast customer service! Highly ...,5
1,"Still setting up my store, and after initially...",5
2,"This is an excellent search app, which they ha...",5
3,"A+, great great great customer service! thanks...",5
4,"I'm begginig to use this app, the search engin...",5
...,...,...
447111,The app is cool and simple to use which I like...,2
447112,"Such a great thing, and great customer service...",5
447113,Just found Privy! I can't wait to start recei...,5
447114,"Wow, overnight got tons of new email signups a...",5


In [ ]:
#Đánh giá nhãn: 
#Có 447116 đánh giá.
#Các bài đánh giá có xếp hạng sao 4,5 được gắn nhãn là đánh giá tích cực 
#1,2 được dán nhãn là đánh giá tiêu cực. 
#Xóa các bài đánh giá có xếp hạng 3 sao vì chúng được coi là trung lập.

In [11]:
df['rating']=df['rating'].astype(int) #chuyển rating  thành  int
df=df[df['rating']!=3]
df['label']=np.where(df['rating']>=4,1,0) #1-Positve,0-Negative

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Số lượng bài đánh giá theo rating

In [12]:
df['rating'].value_counts()

5    379281
4     36653
1     16893
2      4748
Name: rating, dtype: int64

**Pre-Processing**

Bước đầu tiên là chuyển đổi tất cả các đánh giá từ chữ in hoa thành chữ thường.

In [15]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle
data=df[df['label']==0][:50000]
data=data.append(df[df['label']==1][:50000])
data = data.reset_index(drop=True)
display(data['label'].value_counts())
data

1    50000
0    21641
Name: label, dtype: int64

,Sentence,rating,label
0,If I wanted to write code I wouldn't have look...,1,0
1,This app is so bad. Always a problem and when ...,1,0
2,10MAY16 App started off great with a few extra...,1,0
3,"BE AWARE - SCAM, SCAM Company. \nSCAM, SCAM, S...",1,0
4,I installed the app twice on the same day and ...,1,0
...,...,...,...
71636,Love this app! It looks really nice on our pro...,5,1
71637,Great app and even better support :) It wasn't...,5,1
71638,I'm just writing this review cos i need to ext...,5,1
71639,"Absolutely amazing. Excellent support, very re...",5,1


In [17]:
data['pre_process'] = data['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

Xóa các thẻ HTML và URL khỏi các bài đánh giá.

In [18]:
from bs4 import BeautifulSoup
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
import re
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://theinternetcollective.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.aliexpress.com/item/beauty-glazed-18-color-nude-shining-eyeshadow-palette-makeup-glitter-pigment-smoky-eye-shadow-pallete-waterproof/32953735368.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://maxiwholesale.net/" looks like a URL. Beautiful Soup is not an HTTP client. You should 

Chuyển đổi các từ viết tắt thành chữ bt

In [19]:
def contractions(s):
 s = re.sub(r"won't", "will not",s)
 s = re.sub(r"would't", "would not",s)
 s = re.sub(r"could't", "could not",s)
 s = re.sub(r"\'d", " would",s)
 s = re.sub(r"can\'t", "can not",s)
 s = re.sub(r"n\'t", " not", s)
 s= re.sub(r"\'re", " are", s)
 s = re.sub(r"\'s", " is", s)
 s = re.sub(r"\'ll", " will", s)
 s = re.sub(r"\'t", " not", s)
 s = re.sub(r"\'ve", " have", s)
 s = re.sub(r"\'m", " am", s)
 return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

Xóa các ký tự đặc biệt

In [21]:
import nltk
nltk.download('punkt')
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Xóa các khoảng trắng thừa giữa các từ

In [22]:
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(' +', ' ', x))

Loại bỏ các từ dừng bằng cách sử dụng gói NLTK


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Lemmatization bằng wordnet lemmatizer

In [26]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [27]:
data

,Sentence,rating,label,pre_process
0,If I wanted to write code I wouldn't have look...,1,0,wanted write code would looked app store app r...
1,This app is so bad. Always a problem and when ...,1,0,app bad always problem contacting ebay link la...
2,10MAY16 App started off great with a few extra...,1,0,may app started great extra sale month sale co...
3,"BE AWARE - SCAM, SCAM Company. \nSCAM, SCAM, S...",1,0,aware scam scam company scam scam scam ordered...
4,I installed the app twice on the same day and ...,1,0,installed app twice day uninstalled day charge...
...,...,...,...,...
71636,Love this app! It looks really nice on our pro...,5,1,love app look really nice product page support...
71637,Great app and even better support :) It wasn't...,5,1,great app even better support working exactly ...
71638,I'm just writing this review cos i need to ext...,5,1,writing review co need extend trial though mus...
71639,"Absolutely amazing. Excellent support, very re...",5,1,absolutely amazing excellent support responsiv...


Chia Dữ liệu thành Training  và Testing

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['pre_process'], data['label'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (53730,) (53730,) Test:  ((17911,), (17911,))


**TF*IDF Vectorizer**

In [29]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


**Sử dụng SVM**

Triển khai SVM  sklearn để phân loại

In [30]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

Kết hợp dữ liệu được Training vào modal

In [31]:
clf.fit(tf_x_train,Y_train)

LinearSVC(random_state=0)

In [ ]:
#Dự đoán

In [32]:
y_test_pred=clf.predict(tf_x_test)

In [ ]:
#Phân tích kết quả

In [33]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [34]:
report

{'0': {'f1-score': 0.9020081814801041,
  'precision': 0.9175335729146964,
  'recall': 0.8869994514536478,
  'support': 5469},
 '1': {'f1-score': 0.9579510093353547,
  'precision': 0.9510456273764258,
  'recall': 0.9649574023468895,
  'support': 12442},
 'accuracy': 0.9411534811009994,
 'macro avg': {'f1-score': 0.9299795954077295,
  'precision': 0.9342896001455612,
  'recall': 0.9259784269002687,
  'support': 17911},
 'weighted avg': {'f1-score': 0.9408692536801502,
  'precision': 0.9408129532738521,
  'recall': 0.9411534811009994,
  'support': 17911}}

**SVM: 95,1%**

In [ ]:
#Hồi quy tuyến tính

**Hồi quy tuyến tính dùng sklearn**

In [36]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver='saga')

In [37]:
#Fit the Training data to the model
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [38]:
#Dự đoán
y_test_pred=clf.predict(tf_x_test)

In [39]:
#Phân tích
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [40]:
report

{'0': {'f1-score': 0.8995847489618725,
  'precision': 0.9295884532865223,
  'recall': 0.871457304808923,
  'support': 5469},
 '1': {'f1-score': 0.9578212954887814,
  'precision': 0.9450093867334167,
  'recall': 0.9709853721266677,
  'support': 12442},
 'accuracy': 0.940595164982413,
 'macro avg': {'f1-score': 0.928703022225327,
  'precision': 0.9372989200099695,
  'recall': 0.9212213384677954,
  'support': 17911},
 'weighted avg': {'f1-score': 0.9400391686976661,
  'precision': 0.9403007113372319,
  'recall': 0.940595164982413,
  'support': 17911}}

**LR 94.5%**